# Licencia Notebook
Predicción de abandono de clientes para Telco

Copyright (C) 2024  Alvarado Ludwig & Pineda Laura

Este programa es software libre: puedes redistribuirlo y/o modificarlo bajo los términos de la Licencia Pública General de GNU, tal como fue publicada por la Free Software Foundation, ya sea la versión 3 de la Licencia, o (a tu elección) cualquier versión posterior.

Este programa se distribuye con la esperanza de que sea útil, pero SIN NINGUNA GARANTÍA; sin siquiera la garantía implícita de COMERCIABILIDAD o APTITUD PARA UN PROPÓSITO PARTICULAR. Consulta la Licencia Pública General de GNU para obtener más detalles.

Debes haber recibido una copia de la Licencia Pública General de GNU junto con este programa. Si no es así, visita https://www.gnu.org/licenses/.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/93/GPLv3_Logo.svg/1920px-GPLv3_Logo.svg.png" alt="GPL" width="400" height="200">


# Importar librerías y leer dataset

In [1]:
import pandas as pd 


In [2]:
df = pd.read_csv('CustomerChurn.csv', delimiter=',')

# Exploración del set de datos